<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Speed Up DataFrame Operations w/ RAPIDS cuDF

## Welcome
A **DataFrame** is a 2-dimensional data structure used to represent data in a tabular format, like a spreadsheet or SQL table. Originally offered through the Python Data Analysis ([pandas](https://pandas.pydata.org/docs/)) library, DataFrames have become very popular for its familiar representation along with a robust set of features that are intuitive and expressive. 

Raw data often needs to be manipulated before it can be used for further purposes such as generating **Business Intelligence**, creating **Dashboard Visualization**, or training **Machine Learning** models. These preprocessing steps can include **filtering**, **merging**, **grouping**, and **aggregating**. 

Below is a typical data processing pipeline: 
<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/flow.png?raw=true' atl='flow' width=1080></p>

According to [studies](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=29f71b266f63), data preparation accounts for ~80% of the work for analysts. This could be due in part to the rapid increase in the size of data as well as the iterative nature of analytics. 

Recognizing this potential bottleneck, NVIDIA created [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) that leverages GPU hardware and software to perform data manipulation tasks with parallel computing, **saving valuable time and resources**. The cuDF library is part of the larger [**RAPIDS**](https://rapids.ai/) data science framework that allows for the execution of **end-to-end analytics pipelines** entirely on GPUs. One of the focus for cuDF and its companion suite of open source software libraries is to provide syntax that is similar to their CPU counterparts, **making it easy to implement**. 

This notebook is intended to demonstrate speedup in data processing by moving common DataFrame operations to the GPU with minimal changes to existing code. 

### Environment Sanity Check
Check the output of `!nvidia-smi` to make sure you've been allocated a RAPIDS supported GPU such as Tesla T4, P4, or P100.

In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


## Interactive Exercise

In [3]:
import numpy as np # for generating sample data

import pandas as df
# import cudf as df
import time # for clocking process times
import matplotlib.pyplot as plt # for visualizing results

class Timer: # creating a Timer helper class to measure execution time
  def __enter__(self):
    self.start=time.perf_counter()
    return self
  def __exit__(self, *args):
    self.end=time.perf_counter()
    self.interval=self.end-self.start

### Loading a Sample Data
We start our demonstration by generating two 2-dimensional arrays of random numbers - we've configured for sizeable arrays at 1MM rows by 50 columns each. Then they are converted to DataFrames using ```pandas.DataFrame()``` or ```cudf.DataFrame()```:

In [4]:
rows=1000000
columns=50

In [5]:
def load_data(): 
  data_a=np.random.randint(0, 100, (rows, columns))
  data_b=np.random.randint(0, 100, (rows, columns))
  dataframe_a=df.DataFrame(data_a, columns=[f'a_{i}' for i in range(columns)])
  dataframe_b=df.DataFrame(data_b, columns=[f'b_{i}' for i in range(columns)])
  return dataframe_a, dataframe_b

with Timer() as process_time: 
  dataframe_a, dataframe_b=load_data()

print(f'The loading process took {process_time.interval:.2f} seconds')
display(dataframe_a.tail(5))
display(dataframe_b.tail(5))

The loading process took 1.09 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,a_40,a_41,a_42,a_43,a_44,a_45,a_46,a_47,a_48,a_49
999995,8,14,96,59,50,83,66,6,86,61,...,49,64,28,57,38,39,40,44,64,65
999996,47,14,5,55,35,34,84,33,46,16,...,80,81,5,51,81,7,60,61,95,58
999997,41,50,14,85,71,16,4,41,45,41,...,14,39,55,84,65,1,80,38,8,49
999998,37,24,99,15,80,61,56,65,0,4,...,63,0,27,43,77,62,8,78,18,88
999999,46,64,98,38,23,37,19,60,98,26,...,79,19,23,18,12,75,41,46,32,54


,b_0,b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
999995,66,70,73,16,44,80,10,0,8,34,...,93,61,27,66,43,43,37,45,58,13
999996,28,52,83,70,7,2,78,54,88,5,...,90,12,48,4,86,73,44,27,65,96
999997,6,90,81,49,22,8,93,17,51,34,...,6,64,87,54,97,16,3,18,23,52
999998,82,4,97,46,20,99,37,0,16,11,...,98,90,15,40,94,34,61,97,68,40
999999,84,6,76,14,8,19,70,26,60,45,...,36,81,41,68,81,20,24,68,48,21


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We created two DataFrames, _dataframe_a_ and _dataframe_b_ that are 1000000 rows by 50 columns (col_1, col_2, ... col_48, col_49) each. 

### Merging Data
Sometimes data can come from multiple sources and need to be merged into one with ```DataFrame.merge()```. For example, a typical retail data storage infrastructure may include a customer table and separate transaction and product tables. Merging the data allows the correct details to be included in a single DataFrame to get the insight needed. 

In [6]:
def merge_data(left_df, right_df):
  combined_df=df.merge(left_df, right_df, left_index=True, right_index=True)
  return combined_df

with Timer() as process_time: 
  combined_df=merge_data(dataframe_a, dataframe_b)

print(f'The merging process took {process_time.interval:.2f} seconds')
display(combined_df.head())

The merging process took 0.99 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
0,97,2,96,34,38,32,38,21,76,56,...,24,16,4,71,50,96,65,48,41,11
1,84,8,67,73,92,60,50,81,6,60,...,27,11,59,65,66,75,56,32,0,45
2,78,93,32,38,57,84,86,85,79,9,...,77,96,34,85,79,88,90,22,37,72
3,50,95,78,1,7,37,56,62,63,92,...,44,5,45,67,73,53,27,33,15,20
4,69,26,53,4,23,93,64,50,7,23,...,90,20,85,47,55,65,22,92,26,80


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We merged two DataFrames, _dataframe_a_ and _dataframe_b_ on their _index_ into one larger DataFrame that is 1000000 rows by 100 columns (a_0, a_1, ..., b_48, b_49). 

### Summarize
Exploring data begins with **descriptive statistics**, which often involves finding the **central tendency** and **dispersion**. They are a quick way to summarize distributions. Measures of central tendency includes the mean, median, and mode - they are used to describe the center of a set of data values. Measures of dispersion include variance and standard deviation - they are used to describe the degree to which data is distributed around the center. We can quickly perform simple descriptive statistics with the ```DataFrame.describe()``` method. 

In [7]:
def summarize(dataframe):
  summary_df=dataframe.describe()
  return summary_df

with Timer() as process_time: 
  summary_df=summarize(combined_df)

print(f'The summarizing process took {process_time.interval:.2f} seconds')
display(summary_df)

The summarizing process took 3.56 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,49.489439,49.484743,49.439016,49.460502,49.510707,49.471768,49.486676,49.479405,49.515361,49.459404,...,49.495931,49.513672,49.527716,49.537462,49.455562,49.465704,49.539515,49.448345,49.495542,49.542387
std,28.850881,28.874711,28.861761,28.894760,28.847835,28.859780,28.858672,28.849611,28.854317,28.882588,...,28.864611,28.875902,28.863449,28.874621,28.858356,28.850571,28.834406,28.877545,28.853907,28.853210
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,24.000000,24.000000,24.000000,25.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,25.000000,25.000000,24.000000,24.000000,25.000000,24.000000,24.000000,25.000000
50%,49.000000,49.000000,49.000000,49.000000,50.000000,49.000000,49.000000,49.000000,50.000000,49.000000,...,49.000000,50.000000,50.000000,50.000000,49.000000,49.000000,50.000000,49.000000,49.000000,50.000000
75%,74.000000,74.000000,74.000000,74.000000,75.000000,74.000000,74.000000,74.000000,75.000000,74.000000,...,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,74.000000,75.000000,75.000000,75.000000
max,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

Since this is a sample data set, we see that each of columns/features (a_0, a_1, ..., b_48, b_49) have 1000000 values with an average ~50 and standard deviation of ~30

### Correlation - Exploring Relationships
We might be interested in finding relationships/dependencies between two or more variables through their correlation with ```DataFrame.corr()```. Correlation is a number between -1 and 1 that describes the strength of the association between two variables. Two variables with a correlation of 1 suggests that they change together in the same direction while a correlation of -1 suggests that they change together in the opposite direction. 

In [8]:
def correlation(dataframe): 
  corr_df=dataframe.corr()
  return corr_df

with Timer() as process_time: 
  corr_df=correlation(combined_df)

print(f'The correlation process took {process_time.interval:.2f} seconds')
display(corr_df.head())

The correlation process took 25.88 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
a_0,1.000000,-0.000214,0.001189,0.000443,-0.000263,0.001087,-0.000762,0.000967,-0.001910,-0.000247,...,-0.000382,0.000265,0.000880,-0.000571,0.000010,0.000580,0.000804,0.001512,0.000612,0.000198
a_1,-0.000214,1.000000,0.000125,-0.001992,0.003258,-0.000756,0.000240,0.000037,-0.001288,0.000788,...,0.000684,-0.000565,-0.000030,0.000634,-0.000072,0.000716,-0.002154,-0.000725,0.000795,0.000564
a_2,0.001189,0.000125,1.000000,-0.000446,-0.000327,0.001298,-0.000289,0.000856,-0.001139,0.000563,...,0.000911,0.001624,0.000273,-0.002455,0.000481,0.000010,0.002006,0.001530,0.000637,0.000458
a_3,0.000443,-0.001992,-0.000446,1.000000,0.000080,-0.001704,-0.001780,0.000092,0.001306,-0.000231,...,0.001425,-0.001144,-0.000592,0.001572,0.001063,-0.001538,-0.000911,0.001183,0.000143,-0.000575
a_4,-0.000263,0.003258,-0.000327,0.000080,1.000000,0.002896,0.000372,-0.000232,0.000246,0.000373,...,-0.000246,0.000053,-0.001090,0.001474,0.000807,0.000480,-0.000678,-0.001029,0.001567,0.000081


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting cross tabulation shows that each column/feature (a_0, a_1, ..., b_48, b_49) have a perfect correlation (1) with itself and is not correlated (~0) with each other. 

### Grouping
We can compare subsets of the data to explore the significance of categories and classes with the ```DataFrame.groupby()``` method. We can even group continuous data values into a smaller number of bins with ```pandas.cut()``` or ```cudf.cut()``` to simplify our analysis. The groupings usually follow an aggregation such as mean or count. For example, we can group our data into 5 equidistant bins based on their sequential index. 

In [ ]:
def groupby_summarize(dataframe):
    dataframe['group']=dataframe.index
    dataframe['group']=df.cut(dataframe['group'], 5)
    group_describe_df=dataframe.groupby('group').mean().reset_index(drop=True)
    return group_describe_df

with Timer() as process_time: 
    group_describe_df=groupby_summarize(combined_df)

print(f'The grouping process took {process_time.interval:.2f} seconds')
display(group_describe_df)

<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting DataFrame shows that each group maintains an average of ~50 for each column/feature (a_0, a_1, ..., b_48, b_49) as expected for this sample data. 

### Putting it together
We can measure the total elapsed time for this sample data processing workflow. 

In [ ]:
def pipeline():
    performance={}
    with Timer() as process_time: 
        dataframe_a, dataframe_b=load_data()
    performance['load data']=process_time.interval
    with Timer() as process_time: 
        combined_df=merge_data(dataframe_a, dataframe_b)
    performance['merge data']=process_time.interval
    with Timer() as process_time: 
        summarize(combined_df)
    performance['summarize']=process_time.interval
    with Timer() as process_time: 
        correlation(combined_df)
    performance['correlation']=process_time.interval
    with Timer() as process_time: 
        groupby_summarize(combined_df)
    performance['groupby & summarize']=process_time.interval
    if df.__name__=='cudf': 
        df.DataFrame([performance], index=['gpu']).to_pandas().plot(kind='bar', stacked=True)
    else: 
        df.DataFrame([performance], index=['cpu']).plot(kind='bar', stacked=True)
    return None

### Timing the Pipeline on CPU

In [ ]:
import pandas as df
pipeline()

### Switching to GPU
Traditionally, these tasks are frequently done (as we did) using the popular [**pandas**](https://pandas.pydata.org/) library, which only runs on a single CPU. NVIDIA's [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) library was built with the users in mind - by offering nearly identical syntax to its CPU counterpart, developers only have to make few changes to their existing code to take advantage of its capabilities. 

In [ ]:
import cudf as df

**That's it!** cuDF uses nearly identical syntax to the familiar pandas API. **Brilliant!** It's worth noting that there are some features that are unique to each library, but conviniently there are a lot of overlaps. 

In [ ]:
pipeline()

### Comparing Results
In a trial run, **cuDF** completed the data processing tasks in nearly 10x faster than **pandas**. The expectations is that the speedup will be even more significant as the size of the data becomes largers. Feel free to give it a try by modifying the dimensions of the data above. 

![result](https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/result.png?raw=true)

## Conclusion
Congratulations on completing the notebook! Want to learn more about cuDF and the rest of the RAPIDS framework? Check out the follow-up to this course, [Accelerating End-to-End Data Science Workflows]('https://courses.nvidia.com/courses/course-v1:DLI+S-DS-01+V1/about') or our other online courses at [NVIDIA DLI]('https://www.nvidia.com/en-us/training/online/').